<a href="https://colab.research.google.com/github/AziizBg/retriever/blob/synthetic_data/Retreival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AziizBg/retriever

Cloning into 'retriever'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 257 (delta 75), reused 213 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (257/257), 8.24 MiB | 12.50 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [27]:
%cd retriever

/content/retriever


In [7]:
!pip install -r requirements.txt

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
  Using cached torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.16.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.1.0-cp311-cp311-manylinux1_x86_64.whl.metadata (5.7 kB)
  Using cached https://data.pyg.org/whl/torch-2.1.0%2Bcpu/torch_scatter-2.1.2%2Bpt21cpu-cp311-cp311-linux_x86_64.whl (500 kB)
  Using cached https://data.pyg.org/whl/torch-2.1.0%2Bcpu/torch_sparse-0.6.18%2Bpt21cpu-cp311-cp311-linux_x86_64.whl (1.2 MB)
  Using cached https://data.pyg.org/whl/torch-2.1.0%2Bcpu/torch_cluster-1.6.3%2Bpt21cpu-cp311-cp311-linux_x86_64.whl (753 kB)
  Using cached https://data.pyg.org/whl/torch-2.1.0%2Bcpu/torch_spline_conv-1.2.2%2Bpt21cpu-cp311-cp311-linux_x86_64.whl (210 kB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached peft-0.10.0-py3-none-any

In [6]:
!git checkout synthetic_data

Already on 'synthetic_data'
Your branch is up to date with 'origin/synthetic_data'.


In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch_geometric.data import Data
import json
import gensim
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
from pcst_fast import pcst_fast
import re
from datetime import datetime


# Load embedding modules

In [2]:
pretrained_repo = 'sentence-transformers/all-roberta-large-v1'
batch_size = 1024  # Adjust the batch size as needed

class Dataset(torch.utils.data.Dataset):
    def __init__(self, input_ids=None, attention_mask=None):
        super().__init__()
        self.data = {
            "input_ids": input_ids,
            "att_mask": attention_mask,
        }

    def __len__(self):
        return self.data["input_ids"].size(0)

    def __getitem__(self, index):
        if isinstance(index, torch.Tensor):
            index = index.item()
        batch_data = dict()
        for key in self.data.keys():
            if self.data[key] is not None:
                batch_data[key] = self.data[key][index]
        return batch_data

class Sentence_Transformer(nn.Module):

    def __init__(self, pretrained_repo):
        super(Sentence_Transformer, self).__init__()
        print(f"inherit model weights from {pretrained_repo}")
        self.bert_model = AutoModel.from_pretrained(pretrained_repo)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        data_type = token_embeddings.dtype
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).to(data_type)
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, input_ids, att_mask):
        bert_out = self.bert_model(input_ids=input_ids, attention_mask=att_mask)
        sentence_embeddings = self.mean_pooling(bert_out, att_mask)

        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings

def load_sbert():

    model = Sentence_Transformer(pretrained_repo)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_repo)

    # data parallel
    if torch.cuda.device_count() > 1:
        print(f'Using {torch.cuda.device_count()} GPUs')
        model = nn.DataParallel(model)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return model, tokenizer, device


def sbert_text2embedding(model, tokenizer, device, text):
    if len(text) == 0:
        return torch.zeros((0, 1024))

    encoding = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    dataset = Dataset(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask)

    # DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Placeholder for storing the embeddings
    all_embeddings = []

    # Iterate through batches
    with torch.no_grad():

        for batch in dataloader:
            # Move batch to the appropriate device
            batch = {key: value.to(device) for key, value in batch.items()}

            # Forward pass
            embeddings = model(input_ids=batch["input_ids"], att_mask=batch["att_mask"])

            # Append the embeddings to the list
            all_embeddings.append(embeddings)

    # Concatenate the embeddings from all batches
    all_embeddings = torch.cat(all_embeddings, dim=0).cpu()

    return all_embeddings

load_model = {
    'sbert': load_sbert,
}

load_text2embedding = {
    'sbert': sbert_text2embedding,
}

# Preprocess the data

In [3]:
model_name = 'sbert'
path = 'dataset'
path_nodes = f'{path}/nodes'
path_edges = f'{path}/edges'
path_graphs = f'{path}/graphs'

In [4]:
from pymilvus import MilvusClient, DataType, FieldSchema, CollectionSchema

CLUSTER_ENDPOINT = (
    "https://in03-7a5f9d2a1aa84ef.serverless.gcp-us-west1.cloud.zilliz.com"
)
API_KEY = (
    "a73c79fb1924d05aeb410abc0d5669293cc33be37a123953be640725aa42198ef5c1e499cc07f231977c742ad6e6977c6eddec05"
)

milvus_client = MilvusClient(uri=CLUSTER_ENDPOINT, token=API_KEY)

COLLECTION_NAME = "graph_embeddings"
VECTOR_DIM      = 1024          # SBERT default; change if yours differs
METRIC_TYPE     = "COSINE"     # or "IP" for inner-product

# Drop the collection if it already exists (for a clean slate)
if COLLECTION_NAME in milvus_client.list_collections():
    milvus_client.drop_collection(COLLECTION_NAME)

# Define the schema
schema = CollectionSchema(
    fields=[
        FieldSchema(
            name="graph_id",
            dtype=DataType.INT64,
            is_primary=True,
            auto_id=False
        ),
        FieldSchema(
            name="embedding",
            dtype=DataType.FLOAT_VECTOR,
            dim=VECTOR_DIM
        ),
        FieldSchema(                 # optional metadata field
            name="graph_idx",
            dtype=DataType.INT64
        )
    ],
    description="Mean-pooled SBERT embeddings of knowledge graphs"
)

index_params = milvus_client.prepare_index_params()
index_params.add_index("embedding", index_type="IVF_FLAT", metric_type="COSINE", index_params={"nlist": 64})
milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    consistency_level="Strong",
    index_params=index_params
)

In [ ]:
def preprocessing_step_one():
    # Load the graphs from the JSON file
    with open('graphs.json', 'r') as f:
        graphs = json.load(f)

    # Create directories if they don't exist
    os.makedirs(path_nodes, exist_ok=True)
    os.makedirs(path_edges, exist_ok=True)

    # Process each graph
    for i, triples in enumerate(tqdm(graphs)):
        node_map = {}   # Maps node label → node ID
        edges = []

        for h, r, t in triples:
            h = h.lower()
            t = t.lower()
            if h not in node_map:
                node_map[h] = len(node_map)
            if t not in node_map:
                node_map[t] = len(node_map)
            edges.append({'src': node_map[h], 'edge_attr': r, 'dst': node_map[t]})

        # Convert node map to DataFrame
        nodes_df = pd.DataFrame(
            [{'node_id': v, 'node_attr': k} for k, v in node_map.items()],
            columns=['node_id', 'node_attr']
        )

        # Convert edge list to DataFrame
        edges_df = pd.DataFrame(edges, columns=['src', 'edge_attr', 'dst'])

        # Save to CSV
        nodes_df.to_csv(f'{path_nodes}/{i}.csv', index=False)
        edges_df.to_csv(f'{path_edges}/{i}.csv', index=False)

preprocessing_step_one()

In [ ]:
def preprocessing_step_two():
    print("Loading local knowledge base...")
    with open('graphs.json', 'r') as f:
        graphs = json.load(f)

    model, tokenizer, device = load_model[model_name]()
    text2embedding = load_text2embedding[model_name]

    print("Embedding and storing graphs in milvusDB...")
    os.makedirs(path_graphs, exist_ok=True)

    milvus_vectors = []

    for index in tqdm(range(len(graphs))):
        # --- Load nodes & edges ---
        nodes_path = f'{path_nodes}/{index}.csv'
        edges_path = f'{path_edges}/{index}.csv'
        if not os.path.exists(nodes_path) or not os.path.exists(edges_path):
            print(f'Skipping graph {index} (missing files)')
            continue

        nodes = pd.read_csv(nodes_path)
        edges = pd.read_csv(edges_path)
        nodes.node_attr.fillna("", inplace=True)

        if len(nodes) == 0:
            print(f'Empty graph at index {index}')
            continue

        # --- Embed node and edge attributes ---
        x = text2embedding(model, tokenizer, device, nodes.node_attr.tolist())
        edge_attr = text2embedding(model, tokenizer, device, edges.edge_attr.tolist())
        edge_index = torch.LongTensor([edges.src.tolist(), edges.dst.tolist()])

        # --- Save graph as torch_geometric.Data ---
        pyg_graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(nodes))
        torch.save(pyg_graph, f'{path_graphs}/{index}.pt')

        # --- Compute graph-level embedding (mean of node embeddings) ---
        graph_embedding = torch.mean(x, dim=0).cpu().tolist()

        # --- Store in Milvus format: [graph_id, embedding, index] ---
        milvus_vectors.append({"graph_id": index, "embedding": graph_embedding, "graph_idx": index})

    # --- Final batch insert into Milvus ---
    if milvus_vectors:
        milvus_client.insert(
            collection_name=COLLECTION_NAME,
            data=milvus_vectors,
            auto_id=False
        )
        milvus_client.flush(COLLECTION_NAME)
        print(f"Inserted {len(milvus_vectors)} graph embeddings into Milvus.")
    else:
        print("No graphs were inserted into Milvus.")

preprocessing_step_two()

In [5]:
def retrieval_via_pcst(graph, q_emb, textual_nodes, textual_edges, topk=3, topk_e=3, cost_e=0.5):
    c = 0.01
    if len(textual_nodes) == 0 or len(textual_edges) == 0:
        desc = textual_nodes.to_csv(index=False) + '\n' + textual_edges.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])
        graph = Data(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr, num_nodes=graph.num_nodes)
        return graph, desc

    root = -1  # unrooted
    num_clusters = 1
    pruning = 'gw'
    verbosity_level = 0
    if topk > 0:
        n_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, graph.x)
        topk = min(topk, graph.num_nodes)
        _, topk_n_indices = torch.topk(n_prizes, topk, largest=True)

        n_prizes = torch.zeros_like(n_prizes)
        n_prizes[topk_n_indices] = torch.arange(topk, 0, -1).float()
    else:
        n_prizes = torch.zeros(graph.num_nodes)

    if topk_e > 0:
        e_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, graph.edge_attr)
        topk_e = min(topk_e, e_prizes.unique().size(0))

        topk_e_values, _ = torch.topk(e_prizes.unique(), topk_e, largest=True)
        e_prizes[e_prizes < topk_e_values[-1]] = 0.0
        last_topk_e_value = topk_e
        for k in range(topk_e):
            indices = e_prizes == topk_e_values[k]
            value = min((topk_e-k)/sum(indices), last_topk_e_value)
            e_prizes[indices] = value
            last_topk_e_value = value*(1-c)
        # reduce the cost of the edges such that at least one edge is selected
        cost_e = min(cost_e, e_prizes.max().item()*(1-c/2))
    else:
        e_prizes = torch.zeros(graph.num_edges)

    costs = []
    edges = []
    vritual_n_prizes = []
    virtual_edges = []
    virtual_costs = []
    mapping_n = {}
    mapping_e = {}
    for i, (src, dst) in enumerate(graph.edge_index.T.numpy()):
        prize_e = e_prizes[i]
        if prize_e <= cost_e:
            mapping_e[len(edges)] = i
            edges.append((src, dst))
            costs.append(cost_e - prize_e)
        else:
            virtual_node_id = graph.num_nodes + len(vritual_n_prizes)
            mapping_n[virtual_node_id] = i
            virtual_edges.append((src, virtual_node_id))
            virtual_edges.append((virtual_node_id, dst))
            virtual_costs.append(0)
            virtual_costs.append(0)
            vritual_n_prizes.append(prize_e - cost_e)

    prizes = np.concatenate([n_prizes, np.array(vritual_n_prizes)])
    num_edges = len(edges)
    if len(virtual_costs) > 0:
        costs = np.array(costs+virtual_costs)
        edges = np.array(edges+virtual_edges)

    vertices, edges = pcst_fast(edges, prizes, costs, root, num_clusters, pruning, verbosity_level)

    selected_nodes = vertices[vertices < graph.num_nodes]
    selected_edges = [mapping_e[e] for e in edges if e < num_edges]
    virtual_vertices = vertices[vertices >= graph.num_nodes]
    if len(virtual_vertices) > 0:
        virtual_vertices = vertices[vertices >= graph.num_nodes]
        virtual_edges = [mapping_n[i] for i in virtual_vertices]
        selected_edges = np.array(selected_edges+virtual_edges)

    edge_index = graph.edge_index[:, selected_edges]
    selected_nodes = np.unique(np.concatenate([selected_nodes, edge_index[0].numpy(), edge_index[1].numpy()]))

    n = textual_nodes.iloc[selected_nodes]
    e = textual_edges.iloc[selected_edges]
    desc = n.to_csv(index=False)+'\n'+e.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])

    mapping = {n: i for i, n in enumerate(selected_nodes.tolist())}

    x = graph.x[selected_nodes]
    edge_attr = graph.edge_attr[selected_edges]
    src = [mapping[i] for i in edge_index[0].tolist()]
    dst = [mapping[i] for i in edge_index[1].tolist()]
    edge_index = torch.LongTensor([src, dst])
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(selected_nodes))

    return data, desc

In [49]:
def retreival(question, k=3):
    model, tokenizer, device = load_model[model_name]()
    text2embedding = load_text2embedding[model_name]
    # Encode question
    q_emb = text2embedding(model, tokenizer, device, [question])[0]

    # Ensure collection is loaded before search
    try:
        milvus_client.load_collection(COLLECTION_NAME)
    except Exception as e:
        print(f"Error loading collection: {e}")
        return [], []

    # Perform similarity search in Milvus
    search_results = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=[q_emb.tolist()],
        limit=k,
        search_params={"metric_type": METRIC_TYPE, "params": {}},
        output_fields=["graph_idx"]
    )

    # Extract graph indices from results
    hits = search_results[0]
    graph_indices = [hit["entity"]["graph_idx"] for hit in hits]

    # Collect subgraphs and descriptions
    sub_graphs = []
    descriptions = []

    for index in tqdm(graph_indices, desc="Retrieving subgraphs"):
        nodes_path = f'{path_nodes}/{index}.csv'
        edges_path = f'{path_edges}/{index}.csv'
        graph_path = f'{path_graphs}/{index}.pt'

        if not (os.path.exists(nodes_path) and os.path.exists(edges_path) and os.path.exists(graph_path)):
            print(f"Missing data for graph {index}")
            continue

        nodes = pd.read_csv(nodes_path)
        edges = pd.read_csv(edges_path)
        if len(nodes) == 0:
            print(f"Empty graph at index {index}")
            continue

        graph = torch.load(graph_path)

        # Apply your custom retrieval logic (must be defined elsewhere)
        subg, desc = retrieval_via_pcst(
            graph=graph,
            q_emb=q_emb,
            textual_nodes=nodes,
            textual_edges=edges,
            topk=3,
            topk_e=5,
            cost_e=0.5
        )

        sub_graphs.append(subg)
        descriptions.append(desc)

    return sub_graphs, descriptions

question = "What are the common symptoms of asthma?"
retreival(question, k=3)

inherit model weights from sentence-transformers/all-roberta-large-v1


Retrieving subgraphs: 0it [00:00, ?it/s]


([], [])

In [53]:

def construct_textualize_prompt(description, subgraphs):
    """
    Constructs a prompt for the textualization model using the knowledge graph description.
    """
    # Start with a system message or instruction
    system_message = """You are a helpful assistant that summarizes medical knowledge graphs into coherent text.
Based on the provided knowledge graph nodes and relationships, generate a concise and informative paragraph describing the medical concepts and their connections.
Do not include node IDs or graph numbers in your summary. Be very concise and stick to the subgraphs and descriptions provided. Don't add any new information
"""

    # Format the context from the knowledge graph description
    # Split the description into nodes and edges
    nodes = description[0]
    edges = description[1] if len(description) > 1 else ""

    context = "Here is a medical knowledge graph description:\n"
    context += f"Nodes:\n{nodes}\n"
    if edges:
        context += f"Relationships:\n{edges}\n"

    # Construct the final prompt
    prompt = f"{system_message}\n\n{context}\nGenerate a summary of this knowledge graph:\n\nSummary:"
    print("textualize prompt:", prompt)

    return prompt


In [9]:
from openai import OpenAI

# Initialize OpenAI or NVIDIA client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="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"
)

In [54]:
def construct_rag_prompt(question, context):
    """
    Constructs a prompt for RAG using the textualized knowledge graph descriptions.
    """
    # Start with system message
    system_message = """You are a medical knowledge assistant that helps answer questions based on medical knowledge graphs.
Use only the provided medical context derived from knowledge graphs to answer the question. If the provided context doesn't contain enough information to fully answer the question,
say I don't know. If the context doesn't contain any relevant information, say I can't answer this question. Otherwise, if the context has enough information don't use the phrases I don't know and I can't answer this question.
Be precise and factual in your responses. Format the responses for more clarity.
"""

    # Format the context from textualized descriptions
    context = f"Here is the relevant medical context derived from knowledge graphs:\n{context}"

    # Construct the final prompt
    prompt = f"{system_message}\n\n{context}\nQuestion: {question}\n\nAnswer:"

    return prompt

In [55]:

def textualize(descriptions,sub_graphs):
    """
    Textualizes the context from sub_graphs and descriptions using an LLM API call.

    Args:
        sub_graphs (list): A list of torch_geometric.data.Data objects representing the subgraphs.
        descriptions (list): A list of string descriptions for the subgraphs.

    Returns:
        str: The response from the LLM API call.
    """
    # Construct the prompt for the LLM
    prompt = construct_textualize_prompt(descriptions, sub_graphs)

    try:
        # Make the API call to the LLM
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "user", "content": prompt}
            ],
            model="mistralai/mixtral-8x7b-instruct-v0.1"
        )

        # Extract the response
        llm_response = chat_completion.choices[0].message.content
        print(f"\n\ncontext: ",llm_response)
        return llm_response

    except Exception as e:
        print(f"Error during LLM API call: {e}")
        return "An error occurred while generating the response."

In [56]:
def evaluate_response(question, context, response, label=None, report=False):
    """
    Evaluates the LLM's response based on clarity, exactitude, and context adherence.

    Args:
        question (str): Original question
        context (str): Retrieved knowledge graph context
        response (str): LLM's response to evaluate
        label (str, optional): Ground truth label for comparison
        report (bool): Whether to save evaluation to file

    Returns:
        dict: Evaluation scores and feedback
    """

    if report:
        current_date = datetime.now().strftime("%Y-%m-%d")
        with open(f"evaluation_{current_date}.txt", "a") as f:
            f.write(f"Question: {question}\n")
            f.write(f"Context: {context}\n")
            f.write(f"Response: {response}\n")
            f.write(f"Label: {label}\n")


    evaluation_prompt = f"""You are an expert evaluator of medical knowledge responses. Evaluate the following response based on three criteria:

1. Clarity (0-5): How clear and well-structured is the response? 0 is the worst, 1 is the best.
2. Exactitude (0-5): How accurate and precise is the information provided? 0 is the worst, 1 is the best.
3. Context Adherence (0-5): How well does the response stick to the provided knowledge graphs? 0 is the worst, 1 is the best.
4. Relevance (0-5): How relevant is the retrieved Knowledge Graph Context to the question? 0 is the worst, 1 is the best.
5. Completeness (0-5): How complete and thorough is the response? 0 is the worst, 1 is the best.
6. Logical Flow (0-5): How coherent and well-structured is the response? 0 is the worst, 1 is the best.
7. Uncertainty Handling (0-5): How well does the response acknowledge limitations and uncertainties? 0 is the worst, 1 is the best.


Question: {question}

Knowledge Graph Context:
{context}

Response to Evaluate:
{response}

Provide your evaluation in the following format:
CLARITY: [score]/5 - [brief explanation]
EXACTITUDE: [score]/5 - [brief explanation]
CONTEXT ADHERENCE: [score]/5 - [brief explanation]
RELEVANCE: [score]/5 - [brief explanation]
COMPLETENESS: [score]/5 - [brief explanation]
LOGICAL FLOW: [score]/5 - [brief explanation]
UNCERTAINTY HANDLING: [score]/5 - [brief explanation]
OVERALL FEEDBACK: [average score] and 2-3 sentences summarizing the evaluation]
"""

    if label is not None:
        evaluation_prompt += f"Ground Truth Label: {label}\n"

    # new client (llm api) for evaluation different  meta/llama-3.1-405b-instruct

    evaluation = client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role": "system", "content": "You are an expert evaluator of medical knowledge responses."},
            {"role": "user", "content": evaluation_prompt}
        ],
        temperature=0.2,  # Lower temperature for more consistent evaluation
        top_p=0.95,
        max_tokens=1024,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )

    # print("\n=== Response Evaluation ===\n")

    # Initialize variables to store the complete response
    full_evaluation = ""
    scores = {
        "clarity_score": None,
        "exactitude_score": None,
        "context_adherence_score": None,
        "relevance_score": None,
        "completeness_score": None,
        "logical_flow_score": None,
        "uncertainty_handling_score": None,
        "overall_feedback": None
    }

    for chunk in evaluation:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            # print(content, end="")
            full_evaluation += content

    # Extract scores from the complete evaluation
    scores["clarity_score"] = re.findall(r"CLARITY: (\d+(?:\.\d+)?)", full_evaluation)
    scores["exactitude_score"] = re.findall(r"EXACTITUDE: (\d+(?:\.\d+)?)", full_evaluation)
    scores["context_adherence_score"] = re.findall(r"CONTEXT ADHERENCE: (\d+(?:\.\d+)?)", full_evaluation)
    scores["relevance_score"] = re.findall(r"RELEVANCE: (\d+(?:\.\d+)?)", full_evaluation)
    scores["completeness_score"] = re.findall(r"COMPLETENESS: (\d+(?:\.\d+)?)", full_evaluation)
    scores["logical_flow_score"] = re.findall(r"LOGICAL FLOW: (\d+(?:\.\d+)?)", full_evaluation)
    scores["uncertainty_handling_score"] = re.findall(r"UNCERTAINTY HANDLING: (\d+(?:\.\d+)?)", full_evaluation)
    scores["overall_feedback"] = re.findall(r"OVERALL FEEDBACK: (\d+(?:\.\d+)?)", full_evaluation)

    # if the response is "I don't know" set the scores
    if "I can't answer this question" in response:
        scores["clarity_score"] = 0
        scores["exactitude_score"] = 0
        scores["context_adherence_score"] = 0
        scores["relevance_score"] = 0
        scores["completeness_score"] = 0
        scores["logical_flow_score"] = 0
        scores["uncertainty_handling_score"] = 0
        scores["overall_feedback"] = 0

    # Convert list matches to single values
    for key in scores:
        if scores[key]:
            scores[key] = scores[key][0] if isinstance(scores[key], list) else scores[key]

    # print("\n=== Evaluation Complete ===\n")

    if report:
        with open(f"evaluation_{current_date}.txt", "a") as f:
            f.write("\nScores:\n")
            f.write(json.dumps(scores, indent=2))
            f.write("\n\n" + "="*50 + "\n\n")

    return scores


In [57]:
def generate_response(question):
  sub_graphs, descriptions = retreival(question, k=3)
  context = textualize(descriptions, sub_graphs)
  rag_prompt = construct_rag_prompt(question, context, sub_graphs) # the descriptions are textualized in the construct_rag_prompt function

  completion = client.chat.completions.create(
      model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
      messages=[
          {"role": "system", "content": "You are a medical knowledge assistant."},
          {"role": "user", "content": rag_prompt}
      ],
      temperature=0.4,
      top_p=0.95,
      max_tokens=4096,
      frequency_penalty=0,
      presence_penalty=0,
      stream=True
  )

  #   words to delete from the response:
  to_delete = [
    r"\(Knowledge Graph \d+\)",
    r"Knowledge Graph \d+"
  ]

  # Print the streaming response and save the response
  full_response = ""
  for chunk in completion:
    if chunk.choices[0].delta.content is not None:
      content = chunk.choices[0].delta.content
      for word in to_delete:
          content = re.sub(word, "", content)
      # print(content, end="")
      full_response += content

  scores = evaluate_response(question, descriptions, full_response)
  return full_response, scores


# Example usage:
questions = [
    "How does air pollution impact the treatment or worsening of asthma and COPD symptoms?",
    "How does air pollution impact the treatment or worsening of COPD symptoms?",
    "How does air pollution impact the treatment or worsening of asthma symptoms?",
    "What does asthma mean?",
    "What is the color of Zied's shoes?"
]

for question in questions:
    print(f"\n\nQuestion: {question}\n")
    # Get the response
    full_response, score = generate_response(question)
    print(f"Response: {full_response}\n")
    print(f"Score: {score}\n")



Question: How does air pollution impact the treatment or worsening of asthma and COPD symptoms?

inherit model weights from sentence-transformers/all-roberta-large-v1


Retrieving subgraphs: 0it [00:00, ?it/s]


IndexError: list index out of range

## Benchmarking

In [28]:
from scripts.load_qa_data import load_qa

all_questions, all_answers= load_qa()
# all_questions

loading file  generated_qa/all_covid_data_qa.json
loading file  generated_qa/all_asthma_data_qa.json
loading file  generated_qa/all_pneumonia_data_qa.json
loading file  generated_qa/all_copcd_data_qa.json
loading file  generated_qa/all_tuberculosis_data_qa.json


In [32]:
sum_score = 0
for i, question in enumerate(all_questions):
    response = generate_response(question)
    sub_graphs, descriptions = retreival(question, k=3)
    scores = evaluate_response(question, descriptions, response, all_answers[i], True )
    sum_score += float(scores["overall_feedback"])
    # print(sum_score)
average = sum_score / len(all_questions)
print(average)




inherit model weights from sentence-transformers/all-roberta-large-v1


Retrieving subgraphs: 0it [00:00, ?it/s]


inherit model weights from sentence-transformers/all-roberta-large-v1


Retrieving subgraphs: 0it [00:00, ?it/s]


AttributeError: 'list' object has no attribute 'split'